In [ ]:
import pickle
from skimage import io
import copy
import time

import sys
sys.path.append('/Users/jerrytang/.virtualenvs/cv/lib/python2.7/site-packages')

import matplotlib.pyplot as plt
%matplotlib inline

import helpers
import cluster
import evaluation
import registration

In [ ]:
import gmmreg

## Parameters

In [ ]:
files = ['/Users/jerrytang/summer2k17/hydra/data/gt1.csv', '/Users/jerrytang/summer2k17/hydra/data/gt2.csv', '/Users/jerrytang/summer2k17/hydra/data/gt3.csv', '/Users/jerrytang/summer2k17/hydra/data/gt4.csv']
video_path = '/Users/jerrytang/summer2k17/uw_hydra_stack/stk_0001_Substack (1-5000).tif'
# res is 2 for 512 images, 1 for 1024
res = 2 

## Helper Functions

In [ ]:
# load ground truth tracks
def ground_truth_tracks(path):
    df = pd.read_csv(path)
    neuron_count = int(np.nanmax(np.asarray(df[df.columns[0]]) + 1))
    res = [{} for _ in xrange(neuron_count)]
    for i in range(neuron_count):
        mapping = df[df[df.columns[0]] == i]
        for row in mapping.itertuples():
            res[i][row[2] - 1] = (row[3]/2.0, row[4]/2.0)
    return res

In [ ]:
# map detected data to ground truth data space by euclidean distance
def consistent_coords(detected_data, ground_truth_data, time):
    consistent = []
    count = 0
    for row in detected_data[0]:
        coords = (row[0], row[1])
        dists = list(map(lambda x: helpers.eucl(x, coords), ground_truth_data))
        match_index = dists.index(min(dists))
        match = ground_truth_data[match_index]
        if min(dists) < 5:
            cp = copy.copy(row)
            cp[0] = match[0]
            cp[1] = match[1]
            consistent.append(cp)
    return consistent

# get ground truth neurons for a given time frame
def ground_truth_time(ground_truth, time):
    time_list = []
    for neuron in ground_truth:
        time_list.append(neuron[time])
    return time_list

In [ ]:
# convert ground truth data at a given time frame into map from current time to next time
def ground_truth_map(ground_truth, time):
    m = {}
    for neuron in ground_truth:
        m[neuron[time]] = neuron[time + 1]
    return m

In [ ]:
# given a point set registration from t to t + 1 return what percentage was mapped correctly 
def error(reg_map, ground_truth_map):
    count = len(reg_map.keys())
    right = 0.0
    for x in reg_map.keys():
        if reg_map[x] == ground_truth_map[x]:
            right += 1
    return right, count

In [ ]:
# get fiducials of a frame with a max intensity cutoff
# eventually planning to incorporate size into fiducial calculation as well
# but size is detected as accurately ie the size is a better representation of the
# detected spot size than it is of the actual neuron size, an issue not presented by
# max intensity detections
def get_fiducials(spots, cutoff):
    fid = []
    for spot in spots:
        if spot[3] > cutoff: 
            fid.append(spot)
    return np.asarray(fid)

In [ ]:
# for sorting by max intensity
def getKey(item):
    return item[3]

# get top x percent of fiducials by intensity
def get_fiducials_percentage(spots, percentage):
    total = int(percentage * len(spots))
    fid = sorted(spots, key=getKey, reverse=True)[0:total]
    return np.asarray(fid)

In [ ]:
# load files adapted to current schema of ground truth data
def load_files(paths):
    vid = []
    for path in paths:
        vid.extend(load_file(path))
    print 'video loaded'
    return vid

def load_file(path):
    df = pd.read_csv(path)
    vid = []
    frame_count = int(np.nanmax(np.asarray(df['t']) + 1))
    for i in range(frame_count):
        frame = df[df.t == i]
        coords = frame[['x', 'y', 'Surface', 'max intensity']]
        tuples = [tuple(x) for x in coords.values]
        if len(tuples) != 0:
            vid.append(np.asarray(tuples))
    print 'loaded %d frames' % len(vid)
    return vid

## Initialization

In [ ]:
import pandas as pd
import numpy as np
full = load_files(files)

In [ ]:
vid = io.imread(video_path)[0:199,]

In [ ]:
ground_truth = ground_truth_tracks("/Users/jerrytang/summer2k17/hydra/data/ground_truth.csv")

## GMM Parameters Grid Search

In [ ]:
def gmm_psr(frame, reference, param):
    m = frame[:, [0, 1]]
    s = reference[:, [0, 1]]
    m_info = [frame[:, 2], frame[:, 3]]
    s_info = [reference[:, 2], reference[:, 3]]
    model, scene, after = gmmreg.test('/Users/jerrytang/summer2k17/hydra/data/hydra_config.ini', m, s, m_info, s_info, False, param)
    return model, scene, after

def evaluate_percentage(full, ground_truth, param, percentage): 
    test_frames = [0, 60, 120, 180]
    total_right = 0.0
    total_count = 0.0
    for f in test_frames:
        now_all = consistent_coords(full, ground_truth_time(ground_truth, f), f)
        next_all = consistent_coords(full, ground_truth_time(ground_truth, f + 1), f + 1)
        now_fid = get_fiducials_percentage(now_all, percentage)
        next_fid = get_fiducials_percentage(next_all, percentage)

        m, s, a = gmm_psr(now_fid, next_fid, [param])
        m = now_fid[:, [0, 1]]
        s = next_fid[:, [0, 1]]

        mapping = {}
        for i in range(len(m)):
            dists = list(map(lambda x: helpers.eucl(x, a[i]), s))
            match_index = dists.index(min(dists))
            match = s[match_index]
            mapping[tuple(m[i])] = tuple(match)

        g = ground_truth_map(ground_truth, f)

        right, count = error(mapping, g)
        total_right += right
        total_count += count
    return total_right / total_count

def distance_registration(full, ground_truth, percentage):
    test_frames = [0, 60, 120, 180]
    total_right = 0.0
    total_count = 0.0
    for f in test_frames:
        now_all = consistent_coords(full, ground_truth_time(ground_truth, f), f)
        next_all = consistent_coords(full, ground_truth_time(ground_truth, f + 1), f + 1)
        now_fid = get_fiducials_percentage(now_all, percentage)
        next_fid = get_fiducials_percentage(next_all, percentage)

        m = now_fid[:, [0, 1]]
        s = next_fid[:, [0, 1]]

        mapping = {}
        for i in range(len(m)):
            dists = list(map(lambda x: helpers.eucl(x, m[i]), s))
            match_index = dists.index(min(dists))
            match = s[match_index]
            mapping[tuple(m[i])] = tuple(match)

        g = ground_truth_map(ground_truth, f)

        right, count = error(mapping, g)
        total_right += right
        total_count += count
    return total_right / total_count

In [ ]:
percents = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
params = [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]

In [ ]:
res = np.zeros([len(percents), len(params)])
for i in range(len(percents)):
    for j in range(len(params)):
        res[i][j] = evaluate_percentage(full, ground_truth, params[j], percents[i])

In [ ]:
# plot 3d graph of accuracy as a function of percentage and param
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x = percents
y = params
X, Y = np.meshgrid(x, y)
zs = res
Z = zs.reshape(X.shape)

ax.plot_surface(X, Y, Z)

ax.set_xlabel('Fiducials Percentage')
ax.set_ylabel('Param')
ax.set_zlabel('Registration Accuracy')

plt.show()

In [ ]:
# plot graph of gmm accuracy (from optimal param given each percentage) as a function of percentage 
results_gmm = []
for i in range(len(percents)):
    results_gmm.append(np.max(res[i, :]))
plt.scatter(percents, results_gmm)
plt.xlabel('percentage of fiducials chosen')
plt.ylabel('accuracy')

In [ ]:
# plot graph of distance accuracy as a function of percentage 
results_distance = []
for i in range(len(percents)):
    results_distance.append(distance_registration(full, ground_truth, percents[i]))
plt.scatter(percents, results_distance)
plt.xlabel('percentage of fiducials chosen')
plt.ylabel('accuracy')

## Time Evaluation

In [ ]:
def evaluate_timing(full, ground_truth, percentage): 
    test_frames = [0, 60, 120, 180]
    total_time = 0.0
    for f in test_frames:
        now_all = consistent_coords(full, ground_truth_time(ground_truth, f), f)
        next_all = consistent_coords(full, ground_truth_time(ground_truth, f + 1), f + 1)
        now_fid = get_fiducials_percentage(now_all, percentage)
        next_fid = get_fiducials_percentage(next_all, percentage)

        start_time = time.time()
        m, s, a = gmm_psr(now_fid, next_fid, [2.0])
        total_time += time.time() - start_time
    return total_time / len(test_frames)

In [ ]:
percents_timing = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
results_timing = []
for i in range(len(percents_timing)):
    results_timing.append(evaluate_timing(full, ground_truth, percents_timing[i]))

In [ ]:
plt.scatter(percents_timing, results_timing)
plt.xlabel('percentage of fiducials chosen')
plt.ylabel('seconds')